## Transfer learning attempt 

In [1]:
from gensim.models import Word2Vec
import gensim.downloader as api

v2w_model = None
try:
  v2w_model = gensim.models.Keyedvectors.load('./w2vecmodel.mod')
  print("w2v Model Successfully loaded")
except:
  v2w_model = api.load('word2vec-google-news-300')
  v2w_model.save("./w2vecmodel.mod")
  print("w2v Model Saved")

w2vec_embedding_size = len(v2w_model['pc']) 

[=================================================-] 99.9% 1661.5/1662.8MB downloaded
w2v Model Saved


In [6]:
import pdfplumber
pdf = pdfplumber.open('/Users/tpooja/Downloads/Test_llm1.pdf')
page = pdf.pages[0]
#page1 = pdf.pages[1]
pdf_txt = page.extract_text() #+ page1.extract_text()
pdf.close()

In [7]:
import nltk
nltk.download('punkt')
tokens = nltk.sent_tokenize(pdf_txt)
for t in tokens:
    print(t, "\n")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tpooja\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


Read Skillfully
The Ants and the Grasshopper
CCSSR1: Read closely to determine what the text says explicitly and to make logical inferences from it; cite specific
textual evidence when writing or speaking to support conclusions drawn from the text. 

Includes questions you can ask about any story. 

Story Source: Public Domain, adapted by Center for Urban Education
It is wintertime. 

The weather is cold, but the ants are doing fine. 

They have their warm
home, and they have a lot of food to eat. 

They can wait for the warm weather in spring. 

The
reason the ants have a lot of food is because they worked in the summer. 

So now they have
grain to eat. 

Grain is a kind of plant. 

People plant it and cut it and then they use it to make bread. 

The ants get the grain in summer, because that is when grain grows. 

The ants live in the fields
where the grain grows. 

When the wind blows, some grain falls and the ants rush to get it. 

They
do not eat it right then. 

They save it beca

In [9]:
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

def clean_sentence(sentence, stopwords=False):
  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
  if stopwords:
    sentence = remove_stopwords(sentence)
  return sentence

def get_cleaned_sentences(tokens, stopwords=False):
  cleaned_sentences = []
  for row in tokens:
    cleaned = clean_sentence(row, stopwords)
    cleaned_sentences.append(cleaned)
  return cleaned_sentences

In [15]:
cleaned_sentences = get_cleaned_sentences(tokens, stopwords=True)
cleaned_sentences_with_stopwords = get_cleaned_sentences(tokens, stopwords=False)
sentences = cleaned_sentences_with_stopwords

In [11]:
def getWordVec(word, model):
  samp = model['pc']
  vec = [0]*len(samp)
  try:
    vec = model[word]
  except:
    vec = [0]*len(samp)
  return (vec)


def getPhraseEmbedding(phrase, embeddingmodel):
  samp = getWordVec('computer', embeddingmodel)
  vec = numpy.array([0]*len(samp))
  den = 0;
  for word in phrase.split():
    den = den+1
    vec = vec+numpy.array(getWordVec(word, embeddingmodel))
  return vec.reshape(1, -1)

In [12]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

def retrieveAndPrintFAQAnswer(question_embedding, sentence_embeddings, sentences):
  max_sim = -1
  index_sim = -1
  for index, embedding in enumerate(sentence_embeddings):
    sim = cosine_similarity(embedding, question_embedding)[0][0]
    print(index, sim, sentences[index])
    if sim > max_sim:
      max_sim = sim
      index_sim = index
  
  return index_sim

In [19]:
question = "What lesson did the grasshopper learn?"

In [20]:
import numpy
sent_embeddings = []
for sent in sentences:
  sent_embeddings.append(getPhraseEmbedding(sent, v2w_model))

question_embedding = getPhraseEmbedding(question, v2w_model)
index = retrieveAndPrintFAQAnswer(question_embedding, sent_embeddings, cleaned_sentences_with_stopwords)

0 0.5534168748210311 read skillfully
the ants and the grasshopper
ccssr1 read closely to determine what the text says explicitly and to make logical inferences from it cite specific
textual evidence when writing or speaking to support conclusions drawn from the text
1 0.5010360250483985 includes questions you can ask about any story
2 0.4474563182312811 story source public domain adapted by center for urban education
it is wintertime
3 0.5474807053721459 the weather is cold but the ants are doing fine
4 0.47061132030999675 they have their warm
home and they have a lot of food to eat
5 0.47328884022103124 they can wait for the warm weather in spring
6 0.6058890935416958 the
reason the ants have a lot of food is because they worked in the summer
7 0.49929726519538975 so now they have
grain to eat
8 0.3942916743721668 grain is a kind of plant
9 0.5145836648191955 people plant it and cut it and then they use it to make bread
10 0.5729306327993398 the ants get the grain in summer because th

In [21]:
print("Question: ", question)
print("Answer: ", cleaned_sentences_with_stopwords[index])

Question:  What lesson did the grasshopper learn?
Answer:  the grasshopper had learned a lesson


## Below is the testing for h2ogpte

In [1]:
from h2ogpte import H2OGPTE

client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key='sk-X9MfyZWT1hKFEiNnRSbsnlizxo3kLVKb7RznESmVW6viMDvv',
)

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.5.10`.
You can enable strict version checking by passing strict_version_check=True.


In [63]:
# Not required if collection is already created - Creating a new collection
collection_id = client.create_collection(
    name='Test3',
    description='Fourth test',
)

In [64]:
#If a new collection doesn't have to be created; and we just want to use an already created collection
name = "Test4"
recent_collections = client.list_recent_collections(0, 1000)
for c in recent_collections:
    if c.name == name and c.document_count:
        collection_id = c.id
        break

In [9]:
# Not required if your selected collection already has a document uploaded 
with open('/Users/tpooja/Downloads/LongDocTest_llm.pdf', 'rb') as f:
    doc1 = client.upload('LongDoc.pdf', f)
    # Ingest documents (Creates previews, chunks and embeddings)
client.ingest_uploads(collection_id, [doc1]) #The square brackets around 'doc1' are absolutely essential

Job(id='974e6281-05df-4037-bc6f-3fbe74e36604', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 7, 26, 20, 21, 33, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='ce7097fa95254de1a1fb31f0da35b434', status='Collecting done.'), JobStatus(id='466d2981cfe243269db78fe2d392ac85', status='Indexing done.')], errors=[], last_update_date=datetime.datetime(2024, 7, 26, 20, 22, 21, tzinfo=TzInfo(UTC)), duration='48s', duration_seconds=48.0)

In [65]:
chat_session_id = client.create_chat_session(collection_id)

In [5]:
print([x["base_model"] for x in client.get_llms()])

['mistralai/Mixtral-8x7B-Instruct-v0.1', 'mistralai/Mistral-7B-Instruct-v0.3', 'meta-llama/Meta-Llama-3.1-8B-Instruct', 'meta-llama/Meta-Llama-3.1-70B-Instruct', 'OpenGVLab/InternVL-Chat-V1-5', 'h2oai/h2o-danube3-4b-chat', 'mistral-medium-latest', 'claude-3-5-sonnet-20240620', 'claude-3-haiku-20240307', 'gpt-35-turbo-1106', 'gpt-4-1106-preview', 'gpt-4o', 'gemini-1.5-pro-latest']


In [69]:
import time
llm_model = "gemini-1.5-pro-latest"
toc = time.perf_counter()
with client.connect(chat_session_id) as session:
    reply = session.query(
        'Summarize the document in one paragraph.',
        timeout=300,
        llm = llm_model
    )
    print(llm_model,": ",reply.content)
tic = time.perf_counter()
print("Time taken:",(tic-toc))

gemini-1.5-pro-latest :  According to the document, this study presents a contactless vital sign monitoring system (CVSM) using a near-infrared time-of-flight (ToF) camera for driver monitoring in vehicles. The system extracts heart rate (HR) and respiration rate (RR) by analyzing the intensity and depth information captured by the ToF camera. The study highlights the system's robustness against ambient light changes and its ability to compensate for motion artifacts using depth information. The system's performance was evaluated in various in-vehicle scenarios, demonstrating its potential for real-time driver monitoring and its contribution to safer Advanced Driver-Assistance Systems (ADAS).
Time taken: 58.37232470000163


In [33]:
# Summarizing contents of the document - first we get the chunks in
# Inspect and collect all text chunks
chunks = []
for chunk_id in range(1, 10000): # this range varies as per document size - might need a way to automate this
    try:
        chunk = client.get_chunks(collection_id, [chunk_id])
        print(chunk, flush=True)
        chunks.append(chunk[0].text)
    except:
        break

print(f"Number of chunks: {len(chunks)}", flush=True)

[Chunk(text='CC\nBY\nCitation: Guo, K.; Zhai, T.;\nPurushothama, M.H.; Dobre, A.;\nMeah, S.; Pashollari, E.; Vaish, A.;\nDeWilde, C.; Islam, M.N. Contactless\nVital Sign Monitoring System for\nIn-Vehicle Driver Monitoring Using a\nNear-Infrared Time-of-Flight Camera.\nAppl. Sci. 2022, 12, 4416. https://\ndoi.org/10.3390/app12094416 (Link: https://doi.org/10.3390/app12094416)\nAcademic Editor:\nRubén Usamentiaga\nReceived: 9 March 2022\nAccepted: 24 April 2022\nPublished: 27 April 2022\nPublisher’s Note: MDPI stays neutral\nwith regard to jurisdictional claims in\npublished maps and institutional afﬁl-\niations.\nCopyright:\n© 2022 by the authors.\nLicensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistributed\nunder\nthe\nterms\nand\nconditions of the Creative Commons\nAttribution (CC BY) license (https://\ncreativecommons.org/licenses/by/ (Link: https://creativecommons.org/licenses/by/4.0/)\n4.0/).\n', id=1, name='LongDoc.pdf', size=899, pages='{"version": 0, 

[Chunk(text='2\nOmni Science Inc., Dexter, MI 48130, USA; cdewilde@omnisciinc.com\n*\nCorrespondence: kaiwguo@umich.edu\n†\nCurrent address: EECS Building, 1301 Beal Avenue, Ann Arbor, MI 48109, Abstract: We demonstrate a Contactless Vital Sign Monitoring (CVSM) system and road-test the\nsystem for in-cabin driver monitoring using a near-infrared indirect Time-of-Flight (ToF) camera. The\nCVSM measures both heart rate (HR) and respiration rate (RR) by leveraging the simultaneously\nmeasured grayscale and depth information from a ToF camera. For a camera-based driver monitoring\nsystem (DMS), key challenges from varying background illumination and motion-induced artifacts\nneed to be addressed. In this study, active illumination and depth-based motion compensation are\nused to mitigate these two challenges. For HR measurements, active illumination allows the system\nto work under various lighting conditions, while our depth-based motion compensation has the\nadvantage of directly measur

[Chunk(text='We utilize this correlation to compensate for the motion-induced intensity artifact by\ndifferentiating whether the grayscale intensity change is due to motion or the blood volume\nSci. 2022, 12, 4416. https://doi.org/10.3390/app12094416 (Link: https://doi.org/10.3390/app12094416)\nhttps://www.mdpi.com/journal/applsci (Link: https://www.mdpi.com/journal/applsci)\nAppl. Sci. 2022, 12, 4416\n2 of 23\nchange from the heartbeat. Furthermore, the depth information is also used to estimate the\namount of motion during the measurement period, therefore predicting the quality of the\nHR measurements. Additionally, the depth change from chest wall motion can be used to\ndirectly measure RR, thereby allowing our system to measure both HR and RR with one\nmonolithic sensor. We systematically investigate the performance of our CVSM system by\nperforming HR/RR measurements in various realistic environments with different degrees\nof motion from the roads and the drivers. We study the b

[Chunk(text='With a conventional 2D camera, HR is usually acquired through imaging photo-\nplethysmography (PPG), where the volumetric change of blood ﬂow from the heartbeat\ncauses different amounts of light to be absorbed by the user’s face [17–22]. Several past\nstudies (such as chrominance (CHROM) or independent component analyses (ICA) based\nmethod) have shown the robustness of using a camera to contactlessly measure HR in\ncontrolled environments [23–27] by utilizing the RGB color channels in an RGB camera.\nCompared to measuring HR and RR in a controlled indoor environment, additional\nchallenges from the varying illumination as well as the motion artifacts need to be addressed\nwhen measuring HR and RR while driving. In a moving vehicle, the illumination level\ncan vary signiﬁcantly based on factors such as weather or time of day. Any imaging-based\nsensor will need to be unaffected by the ambient light. Secondly, during driving, both\nthe passive motions from the road or acti

[Chunk(text='Furthermore, even though RR has been shown to be correlated with the drowsiness\nor stress level of the driver [8], the existing camera-based system usually does not have\nan adequate signal quality to extract RR from the HR signal [29,30]. To circumvent the\nlimitation of low signal quality, in this study, we use the ToF camera to measure RR directly\nby measuring the chest wall displacement [28,31], avoiding the potential complexity of\nfusing the data from the radar sensor and the camera, respectively, and reducing the cost\nfor the entire system.\nTo the best of our knowledge, this is the ﬁrst study where a ToF camera is used to\nsimultaneously perform HR and RR measurements in a vehicle. Indirect ToF cameras are\nbecoming increasingly popular on various platforms and are already deployed in vehicles\nfor functions such as Smart Restraint Control Systems (Smart-RCS). Given the beneﬁts\nbrought by the active illumination and depth information, our CVSM system would enab

...

[Chunk(text='As for\nthe RR measurement, even though the 850nm illumination does not penetrate as deep as a\nmillimeter-wave radar, our CVSM system is still able to measure the respiration pattern\nof the chest wall when the user is wearing various types of clothing. With its robustness\nagainst both ambient light and motion artifacts, the ToF-based CVSM system could be a\nkey enabler in low-cost and compact driver/occupant monitoring systems in the vehicle.\nThe physiological information measured by the system could one day empower a safer and\nmore functional ADAS system.\nAuthor Contributions: Conceptualization, K.G., M.N.I., C.D. and T.Z.; methodology, K.G., T.Z.\nand E.P.; software, K.G., M.H.P., A.V. and S.M.; validation, K.G. and E.P.; data analysis, K.G. and E.P.;\ndata curation, K.G., E.P., M.H.P. and A.D.; writing—original draft preparation, K.G.; writing—review\nand editing, K.G., E.P. and M.N.I.; visualization, K.G.;', id=51, name='LongDoc.pdf', size=922, pages='{"version":

[Chunk(text='[CrossRef] (Link: http://dx.doi.org/10.3390/s20092442) [PubMed] (Link: http://www.ncbi.nlm.nih.gov/pubmed/32344815)\nPatel, M.; Lal, S.K.; Kavanagh, D.; Rossiter, P. Applying neural network analysis on heart rate variability data to assess driver\nfatigue. Expert Syst. Appl. 2011, 38, 7235–7242. [CrossRef] (Link: http://dx.doi.org/10.1016/j.eswa.2010.12.028)\nEuroNCAP. Test and Assessment Protocol-Child Presence Detection. Available online: https://cdn.euroncap.com/media/\n64101/euro-ncap-cpd-test-and-assessment-protocol-v10.pdf (Link: https://cdn.euroncap.com/media/64101/ euro-ncap-cpd-test-and-assessment-protocol-v10.pdf) (accessed on 27 February Vicente, J.; Laguna, P.; Bartra, A.; Bailón, R. Drowsiness detection using heart rate variability. Med Eng. Comput.', id=54, name='LongDoc.pdf', size=771, pages='{"version": 0, "selections": [{"page": 21, "block": 9, "polygons": [{"x": 118.0, "y": 596.1586}, {"x": 199.7735, "y": 596.1586}, {"x": 199.7735, "y": 605.1246}, {"x": 1

[Chunk(text='[CrossRef] (Link: http://dx.doi.org/10.1109/JTEHM.2014.2365776) [PubMed] (Link: http://www.ncbi.nlm.nih.gov/pubmed/27170871)\nZhao, P.; Lu, C.X.; Wang, B.; Chen, C.; Xie, L.; Wang, M.; Trigoni, N.; Markham, A. Heart rate sensing with a robot mounted\nmmwave In Proceedings of the 2020 IEEE International Conference on Robotics and Automation (ICRA), Virtual\nConfernece, 31 May–31 August 2020; pp. 2812–2818.\nVerkruysse, W.; Svaasand, L.O.; Nelson, J.S. Remote plethysmographic imaging using ambient Opt. Express 2008,\n16, 21434–21445. [CrossRef] (Link: http://dx.doi.org/10.1364/OE.16.021434) [PubMed] (Link: http://www.ncbi.nlm.nih.gov/pubmed/19104573)\n', id=58, name='LongDoc.pdf', size=658, pages='{"version": 0, "selections": [{"page": 22, "block": 1, "polygons": [{"x": 264.0, "y": 189.9807}, {"x": 347.3055, "y": 189.9807}, {"x": 347.3055, "y": 198.9467}, {"x": 346.0, "y": 198.9467}, {"x": 346.0, "y": 201.4191}, {"x": 559.2551, "y": 201.4191}, {"x": 559.2551, "y": 210.4743},

[Chunk(text='Watkinson, P.J. An assessment of algorithms to estimate\nrespiratory rate from the electrocardiogram and Physiol. Meas. 2016, 37, 610. [CrossRef] (Link: http://dx.doi.org/10.1088/0967-3334/37/4/610) [PubMed] (Link: http://www.ncbi.nlm.nih.gov/pubmed/27027672)\n30.\nvan Gastel, M.; Stuijk, S.; de Haan, G. Robust respiration detection from remote Biomed. Opt. Express\n2016, 7, 4941–4957. [CrossRef] (Link: http://dx.doi.org/10.1364/BOE.7.004941) [PubMed] (Link: http://www.ncbi.nlm.nih.gov/pubmed/28018717)\n31.\nGleichauf, J.; Herrmann, S.; Hennemann, L.; Krauss, H.; Nitschke, J.; Renner, P.; Niebler, C.; Koelpin, A. Automated non-contact\nrespiratory rate monitoring of neonates based on synchronous evaluation of a 3D Time-of-Flight camera and a microwave\ninterferometric', id=63, name='LongDoc.pdf', size=772, pages='{"version": 0, "selections": [{"page": 22, "block": 1, "polygons": [{"x": 330.0, "y": 535.119}, {"x": 559.2552, "y": 535.119}, {"x": 559.2552, "y": 544.1742}, {"x

Number of chunks: 66


In [35]:
# Create summary
summary = client.summarize_content(
    #pre_prompt_summary="Summarize the content below into a list of bullets.\n",
    text_context_list=chunks,
    prompt_summary="Summarize the above into a couple of paragraphs.",
    llm=llm_model,
)

print("\n==== SUMMARY ====")
#for s in summary.content.split("\n"):
print(summary, flush=True)


==== SUMMARY ====
content='The document is a research article published in the journal Applied Sciences in 2022. The article presents a Contactless Vital Sign Monitoring (CVSM) system for in-cabin heart rate (HR) and respiration rate (RR) measurements using a near-infrared indirect Time-of-Flight (ToF) camera. The system addresses the challenges of varying ambient illumination and motion-induced artifacts by utilizing the active illumination and additional depth information from the ToF camera. The system extracts HR using both the average grayscale intensity and depth information from ROI-1 on the right cheek region, while RR is measured using the average depth information of ROI-2 on the chest. The system has been tested in various scenarios, including monitoring both drivers and passengers while driving on highways and local roads. The results show that the CVSM system is ambient light agnostic and can measure HR and RR with high accuracy.\n\nThe article also discusses the potentia

In [21]:
# Does not prove to be an accurate measure of the BLEU score. The online huggingface web application is much more accurate
from nltk.translate.bleu_score import sentence_bleu
candidate = "The grasshopper learned the importance of balancing play and work. He realized that while playing is fun, it's also necessary to work and save for the future, especially during winter when food is scarce. He decided that in the next summer, he would work and play, just like the ants.".split()
ref = ["The grasshopper learnt that although it is fun to play, all play and no work will lead to hardship and that work needs to be done for one to eat. And so, the next summer he also worked and played like the ants.".split()]
print('BLEU score -> {}'.format(sentence_bleu(ref, candidate, weights=(1,0,0,0))))

BLEU score -> 0.42


In [19]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    'this is a dog'.split(),
    'it is dog'.split(),
    'dog it is'.split(),
    'a dog, it is'.split() 
]
candidate = 'it is a dog'.split()
score = sentence_bleu(reference, candidate, weights=(0,1,0,0))
print(score)

#"""candidate = 'it is a dog'.split()
#print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))"""

1.0
